# LlamaIndex: Building an Agent Reasoning Loop

This example demonstrates reasoning over multiple steps using an agent the integrates with function tooling.

Please reference this [DeepLearning.AI](https://learn.deeplearning.ai/courses/building-agentic-rag-with-llamaindex/lesson/ix5w5/building-an-agent-reasoning-loop) course for more details.  

### Set Environment

In [27]:
from dotenv import load_dotenv
import nest_asyncio
import llama_index
from llama_index.llms.openai import OpenAI
import textwrap
from llama_index.core.agent.workflow import AgentWorkflow
from llama_index.core.agent.workflow import FunctionAgent
from llama_index.core.tools import FunctionTool
from pydantic import BaseModel
from llama_index.core.output_parsers import PydanticOutputParser

from tools_util import get_doc_tools


load_dotenv()
nest_asyncio.apply()
llama_index.core.__version__

llm = OpenAI(model="o4-mini", temperature=0)

def long_print(msg: str):
    wrapped_text = textwrap.fill(msg, width=140, replace_whitespace=False)
    print(wrapped_text)

### Load Data

In [2]:
import httpx
import os

files = ['https://arxiv.org/pdf/2505.10543']

os.makedirs('./data', exist_ok=True)

for idx, f in enumerate(files):
    file_name = f"./data/{f.split("/")[-1]}.pdf"
    if not os.path.exists(file_name):
        r = httpx.get(f, timeout=20)
        with open(file_name, 'wb') as f:
            f.write(r.content)

### Creating An Agent

The agent running is created. This is the orchestrator and task dispatcher for the pipeline. The tasks are dispatched to agent workers that utilize the tools provided. The function calling tool is aware of state built up to this point and decides when the result should be returned to the user.

In [ ]:
vector_tool, summary_tool = get_doc_tools("./data/2505.10543.pdf", '2505_10543')

In [ ]:
summarization_response = summary_tool.call("Please provide a concise sentence that summarizes the document.")

In [47]:
long_print(str(summarization_response))

The document explores the evaluation of large language models on dynamic decision-making tasks, emphasizing the performance differences
between larger and smaller models and the impact of strategic prompting techniques on bridging this gap, while also highlighting the
limitations of current models in emergent reasoning and self-learning.


In [49]:
agent1 = FunctionAgent(
    name="2505_10543_search_agent",
    description=str(summarization_response),
    tools=[vector_tool, summary_tool], 
    llm=llm, 
    system_prompt="You are a helpful assistant that can answer questions about a document.",
    verbose=True
)

In [51]:
response = await agent1.run(user_msg="What is the document about?")
long_print(str(response))

The document presents a study on how well large language models (LLMs) perform reasoning tasks in dynamic environments. It examines and
compares the decision-making abilities of several open-source LLMs under different prompting strategies. Key points include:

• Impact of
model size: Larger models generally perform better at dynamic reasoning tasks.  
• Effectiveness of prompting: Techniques like chain-of-
thought, self-consistency, and reflective prompting can bridge some performance gaps but have limits.  
• Current limitations: LLMs still
struggle with planning, spatial coordination, and long-horizon reasoning.  
• Need for robust solutions: The study argues for approaches
beyond prompting—such as better model architectures or learning paradigms—to overcome these shortcomings.  
• Broader research context: It
also surveys related work on quantitative reasoning, reinforcement learning with language models, video-game playing, instruction following,
and automated planning.

Overall, 

An agentic workflow is created with only one agent. The results should be similar. AgentWorkflow is an orchestrator for agents. A good introduction can be found [here](https://www.llamaindex.ai/blog/introducing-agentworkflow-a-powerful-system-for-building-ai-agent-systems). Per the intro, "AgentWorkflow builds on top of our popular Workflow abstractions to make agents even easier to get up and running. AgentWorkflows take care of coordinating between agents, maintaining state, and more, while still giving you the flexibility and extensibility of Workflows."

In [ ]:
workflow = AgentWorkflow(agents=[agent1])
response = await workflow.run(user_msg="What is the document about?")

In [20]:
wrapped_text = textwrap.fill(str(response), width=140, replace_whitespace=False)
print(wrapped_text)

The document examines how large language models perform in dynamic, decision-making environments and evaluates various prompting strategies
(such as self-reflection, heuristic mutation, and planning) to improve their reasoning. It compares models of different sizes on the SMART
PLAY benchmark, highlights gaps between smaller and larger models, and studies the effectiveness of advanced prompts across tasks. The paper
also surveys related work on using language models for quantitative reasoning, reinforcement-learning agent modeling, video-game play,
grounded instruction following, instruction fine-tuning, prompt programming, text-based game benchmarks, contrastive training, and LLM-based
agents—while emphasizing current limitations in planning, reasoning, spatial coordination, and self-learning.


### Creating a Multi-Agent System

Lets more fully utilize the `AgentWorkflow` class.

In [29]:

class IntegerResult(BaseModel):
    """Data model for a tool output."""
    result: int

def add(x: int, y: int) -> int:
    """Adds two integers together."""
    print('Hit add numbers.')
    return x + y

def multiply(x: int, y: int) -> int:
    """Multiply two integers together."""
    print('Hit multiply numbers.')
    return x ** y ** 2

add_tool = FunctionTool.from_defaults(fn=add, description="A tool to add two integers together.")
multiply_tool = FunctionTool.from_defaults(fn=multiply, description="A tool to multiply two integers together.")

agent2 = FunctionAgent(
    name="add_multiply_agent",
    description="An agent that can add or multiply two numbers.",
    tools=[add_tool, multiply_tool], 
    llm=llm, 
    system_prompt=("Only use the available tools to answer questions. "
                   "Do not evaluate output."
                   "The final response should only contain tool output."),
    output_parser=PydanticOutputParser(output_cls=IntegerResult),
    verbose=True
)

In [30]:
response = await agent2.run('Add the numbers 4 and 5.')
assert int(str(response)) == 9
response = await agent2.run('Multiply the numbers 4 and 5.')
assert int(str(response)) == 4 ** 5 ** 2

In [ ]:
workflow = AgentWorkflow(agents=[agent1, agent2], verbose=True)

In [ ]:
response = await workflow.run(user_msg="Multiply 5 and 6 and explain to me the limitations of LLM advanced prompting strategies.")